Test of the mixture sampling Gibbs proceedure.
* Pick n zetas from the posterior
* generate bimonial samples
* generate a free energy estimate
* generate new n zetas

In [1]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline

from simtk import unit
from harmonic_mixture_sampler import HarmonicSwapper
from free_energy_estimators import BayesianSampler

Initialsing the two harmonic oscillators with different variances:

In [2]:
s = (5.0 , 200.0)
sigma1 = s[0] * unit.angstrom
sigma2 = s[1] * unit.angstrom
true_f =  3*np.log(s[0]/s[1])
print 'True free energy difference = {0:f}'.format( true_f )

True free energy difference = -11.066638


Stating the prior distribution of the free energy difference:

In [3]:
prior_model = 'normal'  # the type of distribution
prior_loc = 0        # the center parameter
prior_spread = 20        # the spread parameter, e.g. standard deviation

Initialising the variables for the algorithm:

In [ ]:
n_zeta_resamples = 1    # the number of biasing potentials sampled per iteration
zetas = []               # Where the biasing potentials will be stored
n_moves = []            # the number of label samples taken
n_success = []          # the number of samples in the second state

n_mixture_attempts = 4000     # The number of labelled mixture samples of the harmonic oscillator
mixture_save_freq = 200        # The frequency that the state lables will be save. Preferable a factor of n_mixture_attempts 
print 'The number of label samples per iteration = {0:f}'.format(1.0 * n_mixture_attempts / mixture_save_freq)
openmm = False                 # Whether to use openmm to sample the harmonic oscillator


#median = [prior_loc]
#lowerq = [norm.cdf(0.25,loc = prior_loc, scale = prior_spread)]
#upperq = []

The number of label samples per iteration = 20.000000


In [ ]:
median = []
lowerq = []
upperq = []

print 'True free energy difference = {0:f}'.format( true_f )    
f_samples = np.random.normal(loc = prior_loc, scale = prior_spread, size = n_zeta_resamples)
for i in range(50):
    # Samples zetas:
    new_zetas = []
    for f in f_samples:
        z = np.random.logistic(loc = f)
        zetas.append(z)
        new_zetas.append(z)

    # Mixture sampling:
    for i in range(len(new_zetas)):
        swapper = HarmonicSwapper(sigma1, sigma2, zeta = [0.0,new_zetas[i]])
        swapper.mixture_sample(niterations = n_mixture_attempts, openmm = openmm, save_freq = mixture_save_freq )
        n_success.append( 1.0*swapper.state_counter )
        
    # Bayesian estimation from binomial data
    sampler = BayesianSampler(zetas = np.array(zetas), nsuccesses = np.array(n_success), nsamples = swapper.nmoves)
    chain = sampler.sample_posterior()
    flat_samples = chain[:, 50:, :].reshape((-1, 1))
    
    # Print the current estimate
    #print 'posterior mean = {0:f} and standard deviation = {1:f}'.format(flat_samples.mean(),flat_samples.std())
    
    # Save data
    median.append( np.percentile(flat_samples,50) )
    lowerq.append( np.percentile(flat_samples,97.5) )
    upperq.append( np.percentile(flat_samples,2.5) )
    
    # Generate new f samples to generate new zetas
    f_samples  = np.random.choice(flat_samples[:,0], size = n_zeta_resamples)

True free energy difference = -11.066638


In [ ]:
x = range(len(median))
plt.plot(x,median,color='k')
plt.axhline(y = true_f,color='red',ls='--')
plt.fill_between(x,lowerq,upperq, facecolor='grey',linewidth=0,alpha=0.3,interpolate=True)
plt.xlabel('Step')
plt.ylabel('Free energy')
plt.show()